# Import garden.db

### 3.1 all plants in db

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('gardening.db')
c = conn.cursor()

import readline

# Connect to the database
conn = sqlite3.connect('gardening.db')
c = conn.cursor()

# Set up readline for console input
readline.parse_and_bind("tab: complete")
readline.set_history_length(1000)

# Define a function to execute a SQL query
def execute_query(query):
    try:
        print("=========================================")
        # Execute the SQL query
        c.execute(query)

        # Get column names
        column_names = [description[0] for description in c.description] if c.description else []

        # Print column names
        for col_name in column_names:
            print(col_name, end="\t")
        if column_names:
            print()  # New line after column names


        results = c.fetchall()

        # Print the results
        if len(results) == 0:
            print("No results found.")
        else:
            for row in results:
                print(row)

        print("=========================================")
        
        # Commit the changes to the database
        conn.commit()

        # Print a message for successful INSERT, UPDATE, or DELETE statements
        if c.rowcount > 0:
            print("Query executed successfully.")

        # Print the last inserted row ID for INSERT statements
        if query.lower().startswith('insert'):
            print("Last inserted row ID:", c.lastrowid)

    except sqlite3.Error as e:
        # Print an error message if the query fails
        print("An error occurred:", e)

# Loop to prompt the user for input
query_lines = []
while True:
    try:
        # Check if it's the beginning of a new command
        if not query_lines:
            prompt = "Enter a SQL query (or 'exit' to quit): \n"
        else:
            prompt = ''

        line = input(prompt)
        
        # If the user just presses Enter without typing anything, consider it as execution time
        if not line.strip() and query_lines:
            query = ' '.join(query_lines)
            query_lines.clear()
            execute_query(query)
        else:
            query_lines.append(line)

    except KeyboardInterrupt:
        # Handle Ctrl+C to exit the loop cleanly
        print("\nExiting...")
        break

    # Check if the user wants to exit
    if line.lower() == 'exit':
        break

    # Handle up arrow to load last query (this remains unchanged, but might need a different approach)
    if line == '':
        query = readline.get_history_item(readline.get_current_history_length())

    # Add the query to the history (this remains unchanged)
    readline.add_history(line)



In [ ]:
query = """
SELECT pname
FROM plant
WHERE ptype IN ('עץ פרי', 'עץ נוי');
"""
execute_query(query)

career-readiness-week-1-Dor-sketch

https://github.com/github-student-developer-pack/career-readiness-week-1-Dor-sketch




In [ ]:
# Close the database connection
conn.close()


# 3 using ext sql

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///gardening.db

# SUBQURRIES

## 3.22 plant with max size using subquiry

In [ ]:
%%sql
SELECT pname
FROM plant
WHERE ptype IN ('עץ פרי', 'עץ נוי');

In [ ]:
%%sql
-- # 3.22 plant with max size using subquiry
-- ALL operator not supported!!!
select pname as tallest_plant
from plant
where pmaxheight >=all(select pmaxheight from plant);


In [ ]:
%%sql
-- 3.22
select pname as tallest_plant
from plant
where pmaxheight = (select max(pmaxheight) from plant);

## 3.23 anames where heights diffrences is the biggest

subqury inside having line

In [ ]:
%%sql
select aname
from plant, plantingmap
where plant.pname = plantingmap.pname
group by aname
having (max(pmaxheight)-min(pmaxheight)) >= all
    (select )

# EXIST - check if there is a table in a subquary
retutn true / FALSE
NEED SUBQUIRRY!!!

a * querry returns  full rows...

In [ ]:
%%sql
-- 3.24 find a plant that is planted in only one anmae
SELECT p1.pname
FROM plantingmap AS p1
WHERE NOT EXISTS (
    SELECT *
    FROM plantingmap AS p2
    WHERE p1.pname = p2.pname
    AND p1.aname <> p2.aname
);

In [ ]:
wrong!!!! - I groped by aname instead of pname!!!!

In [ ]:
%%sql
-- 3.24 again but with grouping
--  maybe count?

select pname
from plantingmap
group by aname
having count (aname) = 1;

In [ ]:
%%sql 
-- trying again

select pname
from plantingmap
group by pname
having count(distinct aname) = 1;

show anames
from the plantingmap
group by anames
where aname not exiest in
    welect pnames from plantingmap as p2
    where p2.aname = 'שדרת הזמיר'
    and p2.aname <> p1.aname


לא קיים בהן צמח שלא קיים בשדרות הזמיר


    The GROUP BY should come after the WHERE clause.
    The exists operator is used to determine the existence of rows in a subquery. It doesn't work directly with column values like in. The subquery with exists should be constructed using a correlated subquery.
    In your subquery, you have written p1.panme which seems to be a typo. It should probably be p1.pname.
    It's generally better to structure your query in a more readable format.

Let me correct these issues and provide you with a new query:

```sql

SELECT DISTINCT p1.aname
FROM plantingmap AS p1
WHERE NOT EXISTS (
    SELECT 1
    FROM plantingmap AS p2
    WHERE p2.aname = 'שדרות הזמיר'
      AND p2.pname NOT IN (
          SELECT pname
          FROM plantingmap
          WHERE aname = p1.aname
      )
);
```
This query selects all aname values where there isn't a plant
 (pname) in 'שדרות הזמיר' that doesn't exist in that aname's list of plants.


In [ ]:
%%sql
--3.25 find anames that have at least all plants from 'שדרות הזמיר'
-- wrong!!! I write "exist" instead of "exists"
select distinct p1.aname
from plantingmap as p1
group by aname
where p1.pname not exists in
    (select p1.panme from plantingmap as p2
    where p2.aname = 'שדרות הזמיר'
    and p2.pname <> p1.pname);


In [ ]:
%%sql

SELECT DISTINCT p1.aname
FROM plantingmap AS p1
WHERE NOT EXISTS (
    SELECT 1
    FROM plantingmap AS p2
    WHERE p2.aname = 'שדרות הזמיר'
    -- wrong !!! and p2.pname <> p1.pname
    AND p2.pname NOT IN (
        SELECT pname
        FROM plantingmap
        where aname = p1.aname
    )
)

In [ ]:
%%sql
-- 3.25
SELECT DISTINCT P.aname
FROM plantingmap AS P
WHERE NOT EXISTS (SELECT 0
                  FROM plantingmap
                  WHERE aname = 'שדרות הזמיר'
                    AND pname NOT IN 
                     (SELECT P1.pname
                      FROM plantingmap as P1
                      where P.aname = P1.aname)
                 );

-- 3.26: 3.24 - find plant that exist in at most 1 anme
-- now with unique

wrong! its possible that a plant wont be in any aname!!!!

In [ ]:
%%sql

select aname
from plantingmap
where unique (select p1.pname
                from plantingmap as p1
                where p1.aname <> aname
                  and p1.pname = pname);

In [ ]:
%%sql

select pname
from plant as P1
where unique (select pname
              from palntingmap as P2
              where P1.pname = P2.pname  );